In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

/tmp/ipykernel_700906/2069967446.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import sys
sys.path.append("/data/phil/lapd-msi/") 
import io
import matplotlib.pyplot as plt
import numpy as np
import datetime
import time
import tables
%matplotlib notebook
%load_ext autoreload
%autoreload 2
from mpl_toolkits import mplot3d
# import animatplot as amp
import scipy.signal as sig
# import bapsflib
import glob
import os
import re
import pickle
from tqdm.notebook import tqdm 
from multiprocessing.pool import ThreadPool
from multiprocessing import Pool
import re
import pathlib
from functools import reduce
import torch
import copy

In [3]:
import modular_ebm

In [4]:
def to_timestamp(string):
    return time.mktime(time.strptime(string, '%Y-%m-%d %H:%M:%S'))

In [5]:
glob_list = sorted(glob.glob("data/downsampled_all/*.npz"))

In [6]:
glob_list

['data/downsampled_all/2021-09-28-16-44-26.npz',
 'data/downsampled_all/2021-10-01-18-17-57.npz',
 'data/downsampled_all/2021-10-12-13-30-15.818047.npz',
 'data/downsampled_all/2021-10-13-03-07-15.252798.npz',
 'data/downsampled_all/2021-10-13-10-10-26.757364.npz',
 'data/downsampled_all/2021-10-13-10-11-36.126027.npz',
 'data/downsampled_all/2021-10-13-10-54-27.373462.npz',
 'data/downsampled_all/2021-10-13-18-33-12.958422.npz',
 'data/downsampled_all/2021-10-14-07-27-10.613072.npz',
 'data/downsampled_all/2021-10-15-01-07-27.673829.npz',
 'data/downsampled_all/2021-10-15-10-56-16.884977.npz',
 'data/downsampled_all/2021-10-15-19-20-53.559339.npz',
 'data/downsampled_all/2021-10-16-14-51-38.688964.npz',
 'data/downsampled_all/2021-10-17-16-34-26.802758.npz',
 'data/downsampled_all/2021-10-18-19-25-54.287189.npz',
 'data/downsampled_all/2021-10-18-23-58-59.193492.npz',
 'data/downsampled_all/2021-10-19-04-32-06.580864.npz',
 'data/downsampled_all/2021-10-19-09-48-27.736192.npz',
 'data

In [7]:
len(glob_list)

1309

In [18]:
glob_list[219:221]

['data/downsampled_all/2022-08-31-19-48-08.624552.npz',
 'data/downsampled_all/Big_LaB6_Upgrade 170 10-isat-plane-p31-He1kG-15ms-4600A-htr2150A.npz']

# Load in data

1512 Bytes/shot

In [20]:
# training_set = [
# #                 'data/downsampled_all/2022-02-01-18-44-27.100661.npz',  # don't use because it's too diverse
#                 'data/downsampled_all/2022-03-14-21-14-20.379509.npz',
#                 'data/downsampled_all/2022-05-18-03-38-37.701144.npz',
#                 'data/downsampled_all/2022-06-18-17-04-50.921828.npz',
#                 'data/downsampled_all/2022-07-18-09-53-11.994800.npz'
#                ]

In [21]:
validation_set = ['data/downsampled_all/' + '2022-03-26-22-29-26.046163.npz',
                  'data/downsampled_all/' + 'Big_LaB6_Upgrade 194 03-Isat-p31-Thompson-Hepuff-EW.npz',
                  'data/downsampled_all/' + 'Carter_Experimental 158 13_Hairpin_Langmuir_plane_800G.npz',
                  'data/downsampled_all/' + 'Chen 78 26_41x1line_C12C14C15_A13A41_400G_puff.npz',
                  'data/downsampled_all/' + 'ICRF_campaign 1632 02_AntennaParams_700G_476MHz_FeedHI_r30cm_0degtilt.npz',
                  'data/downsampled_all/' + 'ICRF_campaign 1732 15_Bdotxlinep23_1kG_476MHz_FeedHI_r30cm_30degtilt.npz',
                  'data/downsampled_all/' + 'ICRF_campaign 1833 02_Phase180_Pos20cm_X31_neProfile.npz',
                  'data/downsampled_all/' + 'ICRF_campaign 1939 40_EB23_East_Line2_91V_400Gto2200G_LmCnfg6_20AmpsV2.npz',
                  'data/downsampled_all/' + 'ICRF_campaign 1983 06_AntParams_HeAr_68E10peakdensity_2kG.npz',
                  'data/downsampled_all/' + 'Koepke 27 20_p24p35_IsatVfloat_xline_MaskSourceShorted_DV150_800G.npz',
                  'data/downsampled_all/' + 'Koepke 95 50_allprobes_By200kHz_Bx196kHz_0V.npz',
                  'data/downsampled_all/' + 'Nonlinear_Alfven_Interactions 2547 22_BC15BC11_A13fscan_1000G_x0y0_17perHe.npz',
                  'data/downsampled_all/' + 'Particle_Transport 52 13_line_valves105V_5500A.npz']

In [22]:
training_set = copy.deepcopy(glob_list)
for f in validation_set:
    print(f)
    training_set.remove(f)

data/downsampled_all/2022-03-26-22-29-26.046163.npz
data/downsampled_all/Big_LaB6_Upgrade 194 03-Isat-p31-Thompson-Hepuff-EW.npz
data/downsampled_all/Carter_Experimental 158 13_Hairpin_Langmuir_plane_800G.npz
data/downsampled_all/Chen 78 26_41x1line_C12C14C15_A13A41_400G_puff.npz
data/downsampled_all/ICRF_campaign 1632 02_AntennaParams_700G_476MHz_FeedHI_r30cm_0degtilt.npz
data/downsampled_all/ICRF_campaign 1732 15_Bdotxlinep23_1kG_476MHz_FeedHI_r30cm_30degtilt.npz
data/downsampled_all/ICRF_campaign 1833 02_Phase180_Pos20cm_X31_neProfile.npz
data/downsampled_all/ICRF_campaign 1939 40_EB23_East_Line2_91V_400Gto2200G_LmCnfg6_20AmpsV2.npz
data/downsampled_all/ICRF_campaign 1983 06_AntParams_HeAr_68E10peakdensity_2kG.npz
data/downsampled_all/Koepke 27 20_p24p35_IsatVfloat_xline_MaskSourceShorted_DV150_800G.npz
data/downsampled_all/Koepke 95 50_allprobes_By200kHz_Bx196kHz_0V.npz
data/downsampled_all/Nonlinear_Alfven_Interactions 2547 22_BC15BC11_A13fscan_1000G_x0y0_17perHe.npz
data/downsamp

In [23]:
len(training_set)

1296

In [24]:
np_files = []
shot_filenames = []
for f in tqdm(training_set):
# for f in validation_set:
    temp = np.load(f)['arr_0']
    np_files.append(temp)
    shot_filenames += ([f] * len(temp))

  0%|          | 0/1296 [00:00<?, ?it/s]

In [38]:
for i in range(1296):
    print(len(np_files[i]))

17681
34955
16384
15064
0
358
16384
16384
16384
16384
16384
16384
10341
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
11454
16384
16384
16384
16384
16384
16384
16384
16384
150
172801
172801
172801
172801
172801
172801
172801
172801
172801
172801
172801
172801
172801
42268
172801
172801
172801
172801
128034
172701
172801
172801
172801
172801
35773
141015
0
0
86401
86401
86401
86401
49

In [29]:
len(np_files[1])

34955

In [37]:
(np.floor(np.linspace(0, 16384 - 1, 40))).astype('int')

array([    0,   420,   840,  1260,  1680,  2100,  2520,  2940,  3360,
        3780,  4200,  4620,  5040,  5461,  5881,  6301,  6721,  7141,
        7561,  7981,  8401,  8821,  9241,  9661, 10081, 10501, 10922,
       11342, 11762, 12182, 12602, 13022, 13442, 13862, 14282, 14702,
       15122, 15542, 15962, 16383])

In [79]:
shots_per_dr = 100

In [80]:
cut_np_files_train = []

for i in tqdm(range(len(training_set))):
    temp_shotcount = len(np_files[i])
    if temp_shotcount == 0:
        continue
    if temp_shotcount > shots_per_dr:
        temp_idx_to_keep = (np.floor(np.linspace(0, temp_shotcount - 1, shots_per_dr))).astype('int')
    else:
        temp_idx_to_keep = np.arange(temp_shotcount)
    
    cut_np_files_train.append(np_files[i][temp_idx_to_keep])

  0%|          | 0/1296 [00:00<?, ?it/s]

In [81]:
cut_np_files_test = []

for i in tqdm(range(len(validation_set))):
    temp_shotcount = len(np_files[i])
    if temp_shotcount == 0:
        continue
    if temp_shotcount > shots_per_dr:
        temp_idx_to_keep = (np.floor(np.linspace(0, temp_shotcount - 1, shots_per_dr))).astype('int')
    else:
        temp_idx_to_keep = np.arange(temp_shotcount)
    
    cut_np_files_test.append(np_files[i][temp_idx_to_keep])

  0%|          | 0/13 [00:00<?, ?it/s]

In [82]:
cut_data_train = np.concatenate(cut_np_files_train)
cut_data_test = np.concatenate(cut_np_files_test)

In [83]:
# all_data = np.concatenate(np_files)

In [84]:
# num_shots = all_data.shape[0]
num_shots_train = cut_data_train.shape[0]
num_shots_test = cut_data_test.shape[0]

In [85]:
cut_data.dtype

dtype([('MSI_timestamp', '<f8'), ('datarun_shotnum', '<u4'), ('datarun_timestamp', '<f8'), ('diode_t0', '<f8'), ('discharge_current', '<f4', (32,)), ('discharge_voltage', '<f4', (32,)), ('interferometer', '<f4', (32,)), ('diode_0', '<f4', (32,)), ('diode_1', '<f4', (32,)), ('diode_2', '<f4', (32,)), ('diode_3', '<f4', (32,)), ('diode_4', '<f4', (32,)), ('magent_profile', '<f4', (64,)), ('pressures', '<f4', (51,))])

In [86]:
num_shots_train

123903

In [87]:
num_shots_test

1200

In [93]:
filters = [np.where(np.mean(cut_data_train['discharge_current'][:, 18:21], axis=1) > 1e3)[0]]
#            np.where(np.mean(cut_data_train['discharge_voltage'][:, 18:21], axis=1) < -20)[0],
#            np.where(np.mean(cut_data_train['diode_0'][:, 18:21], axis=1) > 0.2)[0],
#            np.where(np.mean(cut_data_train['diode_1'][:, 18:21], axis=1) > 0.2)[0],
#            np.where(np.mean(cut_data_train['diode_2'][:, 18:21], axis=1) > 0.2)[0]]
idx_remain = np.arange(num_shots_train)
for idcs in filters:
    idx_remain = np.intersect1d(idx_remain, idcs)

# Filter incomplete/bad shots

In [188]:
data_remain_train = cut_data_train[idx_remain]

In [189]:
len(data_remain_train)

120363

In [271]:
discharge_current_cut = np.array([])
discharge_voltage_cut = np.array([])
interferometer_cut = np.array([])

diode_0_cut = reduce(np.union1d, (np.where(np.mean(data_remain_train['diode_0'][:, 18:21], axis=1) < 0.11)[0],
                                  np.intersect1d(np.where(data_remain_train['diode_t0'] > 400 + 3.72617e9)[0],
                                                 np.where(data_remain_train['diode_t0'] < 1747 + 3.72617e9)[0]),
                                  np.intersect1d(np.where(data_remain_train['diode_t0'] > to_timestamp('2088-2-11 17:37:00'))[0],
                                                 np.where(data_remain_train['diode_t0'] < to_timestamp('2088-2-11 18:38:00'))[0]),
                                  np.where(np.mean(data_remain_train['diode_0'][:, 18:21], axis=1) > 10)[0]
                                 ))

diode_1_cut = np.where(np.mean(data_remain_train['diode_1'][:, 18:21], axis=1) < 0.1)[0]

diode_2_cut = np.array([])

diode_3_cut = np.where(data_remain_train['diode_t0'] < to_timestamp('2088-1-20 14:30:00'))[0]

diode_4_cut = np.where(data_remain_train['diode_t0'] < to_timestamp('2088-1-20 14:30:00'))[0]

magnet_profile_cut = np.array([])
pressures_cut = np.where(data_remain_train['pressures'][:] > 0)[0]

In [273]:
print(diode_0_cut.shape)
print(diode_1_cut.shape)
print(diode_3_cut.shape)
print(diode_4_cut.shape)
print(pressures_cut.shape)

(34806,)
(34118,)
(45093,)
(45093,)
(2024,)


In [274]:
filter_idx = reduce(np.union1d, (diode_0_cut, diode_1_cut, diode_3_cut, diode_4_cut, pressures_cut))

In [275]:
keep_idx = np.ones(len(data_remain_train), dtype=bool)
keep_idx[filter_idx] = 0

In [327]:
data_filtered_train = data_remain_train[keep_idx]

In [328]:
len(data_filtered_train)

73096

In [329]:
dtype = np.dtype([('MSI_timestamp', '<f8'), ('datarun_shotnum', '<u4'), ('datarun_timestamp', '<f8'), ('diode_t0', '<f8'),
                  ('discharge_current', '<f4', (32,)), ('discharge_voltage', '<f4', (32,)), ('interferometer', '<f4', (32,)),
                  ('diode_0', '<f4', (32,)), ('diode_1', '<f4', (32,)),('diode_2', '<f4', (32,)), ('diode_3', '<f4', (32,)),
                  ('diode_4', '<f4', (32,)), ('magnet_profile', '<f4', (64,)), ('pressures', '<f4', (51,))])

In [330]:
data_filtered_train = data_filtered_train.astype(dtype)

## Replace NaN'd pressures with -22, floor

In [331]:
np.nan_to_num(data_filtered_train['pressures'], copy=False, nan=-22)

array([[ -9.408779, -12.5799  , -10.251642, ..., -20.323767, -20.39398 ,
        -20.26351 ],
       [ -9.595988, -13.135204, -10.63295 , ..., -20.472801, -20.496222,
        -20.610119],
       [ -9.538831, -13.306646, -10.533356, ..., -20.37641 , -20.40628 ,
        -20.40319 ],
       ...,
       [ -9.790141, -17.592438, -16.060781, ..., -20.180172, -20.200111,
        -20.243855],
       [ -9.755051, -17.611092, -16.086527, ..., -20.162073, -20.164986,
        -20.217894],
       [ -9.68836 , -17.629755, -16.107742, ..., -20.18512 , -20.15531 ,
        -20.203644]], dtype=float32)

## Clip pressures

In [332]:
data_filtered_train['pressures'] = np.clip(data_filtered_train['pressures'], a_min=-22, a_max=0)

## Fix diode_4 offset

In [333]:
data_filtered_train['diode_4'] = data_filtered_train['diode_4'] - np.mean(data_filtered_train['diode_4'][:, 0:4], axis=1, keepdims=True)

# Scale by mean and ptp

In [340]:
dtype_scale = np.dtype([('MSI_timestamp', '<f8'), ('datarun_shotnum', '<f4'), ('datarun_timestamp', '<f4'), ('diode_t0', '<f4'),
                        ('discharge_current', '<f4'), ('discharge_voltage', '<f4'), ('interferometer', '<f4'),
                        ('diode_0', '<f4'), ('diode_1', '<f4'),('diode_2', '<f4'), ('diode_3', '<f4'),
                        ('diode_4', '<f4'), ('magnet_profile', '<f4'), ('pressures', '<f4')])

In [341]:
data_filtered_train_mean = np.array([0], dtype=dtype_scale)
data_filtered_train_ptp = np.array([0], dtype=dtype_scale)

In [342]:
for n in data_filtered_train.dtype.names:
    data_filtered_train_mean[n] = np.mean(data_filtered_train[n])
    data_filtered_train_ptp[n] = np.ptp(data_filtered_train[n])

In [343]:
data_filtered_train_ptp['pressures']

array([13.531602], dtype=float32)

In [344]:
for n in data_filtered_train.dtype.names:
    print(n + ": ", end='')
    print(np.mean(data_filtered_train[n]))

MSI_timestamp: 3736871368.052691
datarun_shotnum: 2055.639091058334
datarun_timestamp: 3726495869.184805
diode_t0: 3736871373.5568686
discharge_current: 1749.1404
discharge_voltage: -59.018448
interferometer: 0.57081217
diode_0: 3.087305
diode_1: 2.3576303
diode_2: 0.89597005
diode_3: 0.56963444
diode_4: 0.0015848743
magnet_profile: 915.5684
pressures: -18.814716


In [345]:
for n in data_filtered_train.dtype.names:
    print(n + ": ", end='')
    print(np.ptp(data_filtered_train[n]))

MSI_timestamp: 19542387.715307236
datarun_shotnum: 66625
datarun_timestamp: 3745006684.2466097
diode_t0: 19542385.179088593
discharge_current: 11486.8125
discharge_voltage: 198.92627
interferometer: 1.8318902
diode_0: 9.631628
diode_1: 9.921337
diode_2: 9.670884
diode_3: 9.694459
diode_4: 0.10979183
magnet_profile: 3569.5022
pressures: 13.531602


In [192]:
for fieldname in scaling_mult.dtype.names:
    data_remain[fieldname] = data_remain[fieldname] / scaling_mult[fieldname]

In [351]:
data_train = np.zeros(len(data_filtered_train), dtype=dtype)
for n in dtype.names:
    data_train[n] = (data_filtered_train[n] - data_filtered_train_mean[n]) / data_filtered_train_ptp[n]

# Save data

In [354]:
np.savez('data/data-MSI-cut_2025-02-07', signals=data_train, mean=data_filtered_train_mean, ptp=data_filtered_train_ptp)

# Test data

# Plot all shots or summary subset

In [356]:
data_train.dtype

dtype([('MSI_timestamp', '<f8'), ('datarun_shotnum', '<u4'), ('datarun_timestamp', '<f8'), ('diode_t0', '<f8'), ('discharge_current', '<f4', (32,)), ('discharge_voltage', '<f4', (32,)), ('interferometer', '<f4', (32,)), ('diode_0', '<f4', (32,)), ('diode_1', '<f4', (32,)), ('diode_2', '<f4', (32,)), ('diode_3', '<f4', (32,)), ('diode_4', '<f4', (32,)), ('magnet_profile', '<f4', (64,)), ('pressures', '<f4', (51,))])

In [ ]:
name = shot_filenames[0]

data_plot = data_train

disp_shots = 1000 if data_plot.shape[0] > 1000 else data_plot.shape[0]

idx = np.linspace(0, data_plot.shape[0] - 1, disp_shots)
idx = np.around(idx).astype(int)

alpha = np.clip(0.05 * 1000 / disp_shots, 0.0, 1.0)
c_discharge = [0.3, 0.9, 1.0, alpha]
c_interferometer = [0.2, 1.0, 0.3, alpha]
c_diode = [0.2, 1.0, 0.3, alpha]
c_white = [0, 0, 0, alpha]

# 21"x12" for 16:9 aspect ratio on 25" monitor
fig, ax = plt.subplots(3, 3, figsize=(16, 12))
fig.suptitle(name[:-3])
ax[0, 0].text(0.5, 0.95, 'Discharge current', color='black', transform=ax[0, 0].transAxes)
ax[0, 0].plot(np.arange(data_plot['discharge_current'].shape[1])[np.newaxis, :].T,
              data_plot['discharge_current'][idx, :].T,
              color=c_discharge)

ax[1, 0].text(0.5, 0.95, 'Discharge voltage', color='black', transform=ax[1, 0].transAxes)
ax[1, 0].plot(np.arange(data_plot['discharge_voltage'].shape[1])[np.newaxis, :].T,
              data_plot['discharge_voltage'][idx, :].T,
              color=c_discharge)

# ax[2, 0].text(0.5, 0.95, 'Interferometer', color='black', transform=ax[2, 0].transAxes)
# ax[2, 0].plot(np.arange(data_plot['interferometer'].shape[1])[np.newaxis, :].T,
#               data_plot['interferometer'][idx, :].T,
#               color=c_interferometer)

ax[0, 1].text(0.5, 0.95, 'diode 0', color='black', transform=ax[0, 1].transAxes)
ax[0, 1].plot(np.arange(data_plot['diode_0'].shape[1])[np.newaxis, :].T,
              data_plot['diode_0'][idx, :].T,
              c=c_diode)

ax[1, 1].text(0.5, 0.95, 'diode 1', color='black', transform=ax[1, 1].transAxes)
ax[1, 1].plot(np.arange(data_plot['diode_1'].shape[1])[np.newaxis, :].T,
              data_plot['diode_1'][idx, :].T,
              c=c_diode)

ax[2, 1].text(0.5, 0.95, 'diode 2', color='black', transform=ax[2, 1].transAxes)
ax[2, 1].plot(np.arange(data_plot['diode_2'].shape[1])[np.newaxis, :].T,
              data_plot['diode_2'][idx, :].T,
              c=c_diode)

try:
    ax[0, 2].text(0.5, 0.95, 'diode 3', color='black', transform=ax[0, 2].transAxes)
    ax[0, 2].plot(np.arange(data_plot['diode_3'].shape[1])[np.newaxis, :].T,
                  data_plot['diode_3'][idx, :].T,
                  c=c_diode)

    ax[1, 2].text(0.5, 0.95, 'diode 4', color='black', transform=ax[1, 2].transAxes)
    ax[1, 2].plot(np.arange(data_plot['diode_4'].shape[1])[np.newaxis, :].T,
                  data_plot['diode_4'][idx, :].T,
                  c=c_diode)
except:
    print("No north discharge info")

ax[2, 2].text(0.5, 0.95, 'Magnet profile', color='black', transform=ax[2, 2].transAxes)
ax[2, 2].plot(data_plot['magnet_profile'][idx, :].T,
              c=c_white)

ax[2, 0].text(0.5, 0.95, 'Gas pressures (log)', color='black', transform=ax[2, 0].transAxes)
ax[2, 0].scatter(np.arange(51)[np.newaxis, :] + np.random.uniform(low=-0.4, high=0.4, size=(disp_shots, 1)),
                 data_plot['pressures'][idx, :],
                 c=[c_white], s=0.7)
# Fake plot ot get autoscaling to work correctly. Weird bug.
ax[2, 3].plot(np.arange(51)[np.newaxis, :] + np.zeros((disp_shots, 1)),
              data_plot['pressures'][idx, :], alpha=0)
# ax[2, 3].set_yscale('log')

plt.tight_layout()
# plt.savefig("data/summaries/" + name[:-3].replace('data/downsampled_all/', '') + '-traces.png')